# Line indicators

In [11]:
import pandas as pd
import numpy as np
import yfinance as yf
from stock_indicators import indicators, Quote
from lightweight_charts import Chart
import asyncio
import nest_asyncio
from websockets import Close

nest_asyncio.apply()

In [12]:
df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
df.reset_index(inplace=True)

df.head()


C:\Users\jwang\AppData\Local\Temp\ipykernel_12864\4216576817.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('TQQQ', start='2020-01-01', multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-01-02,21.760862,21.760862,21.133238,21.250618,65536000
1,2020-01-03,21.169170,21.559639,20.898476,20.910454,72590000
2,2020-01-06,21.571617,21.578803,20.589454,20.666111,64047600
3,2020-01-07,21.545265,21.746488,21.358415,21.593175,53849600
4,2020-01-08,22.041140,22.345369,21.447051,21.550058,79582400


In [13]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]

In [14]:
ema_results_25 = indicators.get_ema(quotes, 25)
df['EMA 25'] = [r.ema for r in ema_results_25]
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df = df.dropna().reset_index(drop=True)
df.head()

,Date,Close,High,Low,Open,Volume,EMA 25,EMA 12,upper_band,middle_band,lower_band,rsi
0,2020-02-06,25.972181,26.000928,25.251132,25.514639,56566400,23.212258,24.073254,25.548904,23.610919,21.672935,69.315885
1,2020-02-07,25.636805,26.065602,25.421210,25.627224,77026000,23.398761,24.313800,25.831385,23.762915,21.694445,66.386577
2,2020-02-10,26.561478,26.568666,25.368512,25.387676,61624400,23.642047,24.659597,26.274289,23.970366,21.666444,70.134127
3,2020-02-11,26.580637,27.277733,26.369832,26.997456,96885200,23.868092,24.955141,26.676574,24.139609,21.602644,70.208243
4,2020-02-12,27.359180,27.392718,26.853727,27.042972,60804000,24.136638,25.324993,27.192867,24.361314,21.529761,73.126623


In [22]:
if __name__ == '__main__':
    chart = Chart()
    chart.legend(visible=True)

    # Convert the 'Date' column to the 'time' format expected by lightweight-charts
    df['time'] = df['Date'].apply(lambda x: int(x.timestamp()))

    # Calculate the signals
    df['ema_12_prev'] = df['EMA 12'].shift(1)
    df['ema_25_prev'] = df['EMA 25'].shift(1)

    buy_signals = df[
        (df['EMA 12'] > df['EMA 25']) & (df['ema_12_prev'] <= df['ema_25_prev'])
    ]
    sell_signals = df[
        (df['EMA 12'] < df['EMA 25']) & (df['ema_12_prev'] >= df['ema_25_prev'])
    ]

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_label=True)
    ema12_line.set(df[['Date', 'EMA 12']])
    
    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_label=True)
    ema25_line.set(df[['Date', 'EMA 25']])
    # Add markers for signals
    # 'CARETUP' and 'CARETDOWN' are appropriate shapes for buy and sell signals
    for _, row in buy_signals.iterrows():
        chart.marker(
            time=row['Date'],
            position='below',  # Place the buy signal below the bar
            shape='caretUp',  # Use an upward-pointing triangle for buy
            color='green',
            text='BUY'
        )

    for _, row in sell_signals.iterrows():
        chart.marker(
            time=row['time'],
            position='above',  # Place the sell signal above the bar
            shape='caretDown',  # Use a downward-pointing triangle for sell
            color='red',
            text='SELL'
        )
    
    chart.set(df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']])
    
    chart.show(block=True)





In [ ]:
# Draw Bollinger Bands
chart = Chart()
chart.legend(visible=True)

chart.set(df)


upper_line = chart.create_line('UpperBB', color = 'red', width = 1, price_label = True)
upper = pd.DataFrame(columns = ['time', 'UpperBB'])
upper['time'] = df.Date
upper['UpperBB'] = df['upper_band']
upper_line.set(upper.dropna())

middle_line = chart.create_line('MiddleBB', color = 'orange', width = 1, price_label = True)
middle = pd.DataFrame(columns = ['time', 'MiddleBB'])
middle['time'] = df.Date
middle['MiddleBB'] = df['middle_band']
middle_line.set(middle.dropna())

lower_line = chart.create_line('LowerBB', color = 'green', width = 1, price_label = True)
lower = pd.DataFrame(columns = ['time', 'LowerBB'])
lower['time'] = df.Date
lower['LowerBB'] = df['lower_band']
lower_line.set(lower.dropna())

# Show the chart
chart.show(block=True)

In [ ]:
if __name__ == '__main__':
    # Create the main candlestick chart
    chart = Chart(inner_height=0.8, width=1)
    chart.time_scale(visible=False)
    chart.set(df)

    # Create a sub-chart for the RSI indicator
    rsi_chart = chart.create_subchart(height=0.2, width=1, sync=True)
    rsi_line = rsi_chart.create_line('RSI14', color = 'red', width = 1, price_label = True)
    rsi = pd.DataFrame(columns = ['time', 'RSI14'])
    rsi['time'] = df.Date
    rsi['RSI14'] = df['rsi']
    rsi_line.set(rsi)


    chart.show(block=True)